In [1]:
import pandas as pd
import json
import sqlite3
orders = pd.read_csv("orders.csv")
orders.head()
with open("users.json") as f:
    users_data = json.load(f)

users = pd.DataFrame(users_data)
users.head()
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
restaurants.head()
merged1 = pd.merge(
    orders,
    users,
    left_on="user_id",
    right_on="user_id",
    how="left"
)
final_df = pd.merge(
    merged1,
    restaurants,
    left_on="restaurant_id",
    right_on="restaurant_id",
    how="left"
)

final_df.head()
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [2]:
gold_df = final_df[final_df["membership"] == "Gold"]

gold_city_revenue = gold_df.groupby("city")["total_amount"].sum()

gold_city_revenue.sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [3]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [5]:
user_spend = final_df.groupby("user_id")["total_amount"].sum()

high_spenders = user_spend[user_spend > 1000]

len(high_spenders)


2544

In [8]:
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[3.0,3.5,4.0,4.5,5.0],
    labels=["3.0-3.5","3.6-4.0","4.1-4.5","4.6-5.0"]
)
final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


C:\Users\DELL 3410\AppData\Local\Temp\ipykernel_14648\752595886.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


rating_range
4.6-5.0    2197030.75
4.1-4.5    1960326.26
3.0-3.5    1881754.57
3.6-4.0    1717494.41
Name: total_amount, dtype: float64

In [9]:
gold_df = final_df[final_df["membership"] == "Gold"]

gold_df.groupby("city")["total_amount"].mean().sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [12]:
rest_count = final_df.groupby("cuisine")["restaurant_id"].nunique()
rest_count
revenue = final_df.groupby("cuisine")["total_amount"].sum()
revenue
analysis = pd.DataFrame({
    "restaurant_count": rest_count,
    "revenue": revenue
})

analysis.sort_values(by="restaurant_count")


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [13]:
total_orders = len(final_df)
gold_orders = len(final_df[final_df["membership"] == "Gold"])
percentage = (gold_orders / total_orders) * 100
round(percentage)


50

In [23]:

order_counts = final_df.groupby("restaurant_name_x")["order_id"].count()


small_rest = order_counts[order_counts < 20].index


avg_value = final_df.groupby("restaurant_name_x")["total_amount"].mean()


result = avg_value.loc[small_rest].sort_values(ascending=False)

result.head()


restaurant_name_x
Hotel Dhaba Multicuisine             1040.222308
Sri Mess Punjabi                     1029.180833
Ruchi Biryani Punjabi                1002.140625
Sri Delights Pure Veg                 989.467222
Classic Kitchen Family Restaurant     973.167895
Name: total_amount, dtype: float64

In [24]:
combo_revenue = final_df.groupby(
    ["membership","cuisine"]
)["total_amount"].sum().sort_values(ascending=False)

combo_revenue.head()


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Name: total_amount, dtype: float64

In [25]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.quarter
final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


C:\Users\DELL 3410\AppData\Local\Temp\ipykernel_14648\366582611.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [26]:
gold_orders_count = final_df[final_df["membership"] == "Gold"].shape[0]
print("1. Total orders by Gold members:", gold_orders_count)

1. Total orders by Gold members: 4987


In [27]:
hyderabad_revenue = final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
print("2. Total revenue Hyderabad (rounded):", round(hyderabad_revenue))

2. Total revenue Hyderabad (rounded): 1889367


In [28]:
distinct_users = final_df["user_id"].nunique()
print("Distinct users who placed at least one order:", distinct_users)

Distinct users who placed at least one order: 2883


In [29]:
avg_gold_order = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()
print("Average order value for Gold members:", round(avg_gold_order, 2))

Average order value for Gold members: 797.15


In [30]:
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]
print("Orders for restaurants with rating >=4.5:", high_rating_orders)

Orders for restaurants with rating >=4.5: 3374


In [31]:
gold_df = final_df[final_df["membership"] == "Gold"]
top_gold_city = gold_df.groupby("city")["total_amount"].sum().idxmax()
orders_top_gold_city = gold_df[gold_df["city"] == top_gold_city].shape[0]
print("Orders in top revenue city among Gold members:", orders_top_gold_city)

Orders in top revenue city among Gold members: 1337
